## Summary

----

## Imports

In [ ]:
import concurrent.futures
import itertools
import os
import pickle
import shelve
import socket
from datetime import datetime
from pathlib import Path

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import numpy as np
import psutil
import pyarrow as pa
import pyarrow.parquet as pq

## Parameters

In [ ]:
NOTEBOOK_PATH = Path(os.getenv("CI_JOB_NAME", "02_process_pdb_core_validation"))

NOTEBOOK_PATH

In [ ]:
OUTPUT_PATH = Path(os.getenv('OUTPUT_DIR', NOTEBOOK_PATH.name)).resolve()
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH

In [ ]:
if "scinet" in socket.gethostname():
    CPU_COUNT = 40
else:
    CPU_COUNT = max(1, len(os.sched_getaffinity(0)) // 2)

CPU_COUNT

In [ ]:
PDB_DATA_PATH = (
    Path(os.environ["DATAPKG_OUTPUT_DIR"]).joinpath("pdb-ffindex", "2020-01-16", "arrow").resolve(strict=True)
)

PDB_DATA_PATH

In [ ]:
ADJACENCY_NET_DATA_PATH = (
    Path(os.getenv("DATAPKG_OUTPUT_DIR")).joinpath("adjacency-net-v2", "v0.3").resolve(strict=True)
)

ADJACENCY_NET_DATA_PATH

## Workflow

In [ ]:
with PDB_DATA_PATH.joinpath("pdb-list.pickle").open("rb") as fin:
    pdb_list = pickle.load(fin)
    
pdb_data_reader = pa.RecordBatchFileReader(PDB_DATA_PATH.joinpath("pdb-mmcif.arrow"))

assert len(pdb_list) == pdb_data_reader.num_record_batches

In [ ]:
pdb_list[:3]

In [ ]:
output_dir = ADJACENCY_NET_DATA_PATH.joinpath("pdb-core")
output_dir_failed = output_dir.joinpath("failed")

output_dir, output_dir_failed

In [ ]:
def worker(task_id, task_count, progress=False):
    chunk_size = int(np.ceil(len(pdb_list) / task_count))

    task_idx = task_id - 1
    stats = {
        "succeeded": True,
        "pdbs_succeeded": set(),
        "pdbs_failed": set(),
        "pdbs_missing": set(),
        "chains_succeeded_count": 0,
        "error_records": [],
    }

    # Create a subset of PDB structures
    pdb_chunk = pdb_list[task_idx * chunk_size : (task_idx + 1) * chunk_size]
    pdb_chunk_idxs = list(range(task_idx * chunk_size, (task_idx + 1) * chunk_size))
    pdb_chunk_set = set(pdb_chunk)

    # Read succeeded domains
    output_file = output_dir.joinpath(f"pdb-core-{task_id}-{task_count}.arrow")
    if not output_file.with_suffix(".SUCCESS").is_file():
        stats["succeeded"] = False
        return stats

    if output_file.is_file():
        reader = pa.RecordBatchFileReader(output_file)
        for i in tqdm(range(reader.num_record_batches), desc="succeeded", leave=False, disable=not progress):
            pdb_id = reader.get_record_batch(i).column(0)[0].as_py()
            stats["pdbs_succeeded"].add(pdb_id)
            stats["chains_succeeded_count"] += 1
    assert not stats["pdbs_succeeded"] - pdb_chunk_set

    # Read failed pdbs
    output_file_failed = output_dir_failed.joinpath(f"pdb-core-{task_id}-{task_count}-failed.arrow")
    if output_file_failed.is_file():
        reader_failed = pa.RecordBatchFileReader(output_file_failed)
        for i in tqdm(range(reader_failed.num_record_batches), desc="failed", leave=False, disable=not progress):
            error_record = reader_failed.get_record_batch(i).to_pydict()
            pdb_id = error_record["pdb_id"][0]
            stats["pdbs_failed"].add(pdb_id)
            stats["error_records"].append(error_record)
    assert not stats["pdbs_failed"] - pdb_chunk_set

    stats["pdbs_missing"] = pdb_chunk_set - stats["pdbs_succeeded"] - stats["pdbs_failed"]

    return stats

### Calculate statistics for each chunk

In [ ]:
stats_cache_file = OUTPUT_PATH.joinpath("stats.cache")
stats = {}
task_count = 300

with shelve.open(stats_cache_file.as_posix()) as stats_cache:
    for key, stat in stats_cache.items():
        if stat["succeeded"]:
            stats[int(key)] = stat

In [ ]:
with shelve.open(stats_cache_file.as_posix()) as stats_cache:
    with concurrent.futures.ProcessPoolExecutor(CPU_COUNT) as pool:
        task_ids = [i for i in range(1, task_count + 1) if i not in stats]
        futures = pool.map(worker, task_ids, itertools.repeat(task_count))
        for stat, task_id in tqdm(zip(futures, task_ids), total=len(task_ids)):
            stats[task_id] = stat
            stats_cache[str(task_id)] = stats[task_id]

### Plot statistics

In [ ]:
task_idxs = np.arange(300)
num_pdbs_succeeded = np.zeros(300)
num_pdbs_failed = np.zeros(300)
num_pdbs_missing = np.ones(300) * 532.0

for i, task_id in enumerate(sorted(stats)):
    assert task_idxs[i] == task_id - 1

    stat = stats[task_id]
    if stat["succeeded"]:
        num_pdbs_succeeded[i] = len(stat["pdbs_succeeded"])
        num_pdbs_failed[i] = len(stat["pdbs_failed"])
        num_pdbs_missing[i] = len(stat["pdbs_missing"])

In [ ]:
kwargs = {"width": 1.0, "align": "edge"}

with plt.rc_context({"font.size": 14}):
    fg, ax = plt.subplots(figsize=(18, 8))
    ax.bar(task_idxs, num_pdbs_succeeded, label="succeeded", **kwargs)
    ax.bar(task_idxs, num_pdbs_failed, bottom=num_pdbs_succeeded, label="failed", **kwargs)
    ax.bar(task_idxs, num_pdbs_missing, bottom=num_pdbs_succeeded + num_pdbs_failed, label="missing", **kwargs)
    ax.set_xlim(-1, 300)
    ax.set_ylim(0, 580)
    ax.set_xlabel("Chunk index")
    ax.set_ylabel("Number of PDBs")
    ax.legend(loc="upper center", ncol=3)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

In [ ]:
failed_task_ids = [str(task_id) for task_id, data in stats.items() if not data["succeeded"]]

",".join(failed_task_ids)

In [ ]:
for file in NOTEBOOK_PATH.parent.joinpath("01_process_pdb_core", "logs").glob("*.ipynb"):
    parts = file.stem.split("-")
    if parts[0] not in failed_task_ids and parts[-1] not in failed_task_ids:
        file.unlink()

In [ ]:
!ls {NOTEBOOK_PATH}/logs